In [1]:
import pandas as pd
import datetime
import pickle
import os
os.chdir('..')

In [2]:
import pickle
with open('v2/fulloutput_train.pickle','rb') as flight:
     df_flight = pickle.load(flight)
with open('../data/weather_dict.pickle','rb') as weather_data_file:
     df_weather = pickle.load(weather_data_file)

In [3]:
df_flight['origin_state'] = df_flight['origin_city_name'].str[-2:]
df_flight['dest_state'] = df_flight['dest_city_name'].str[-2:]
df_flight.fl_date = pd.to_datetime(df_flight.fl_date)

In [4]:
t = pd.json_normalize(df_weather).apply(lambda x : x[0]['TCAT'])
p = pd.json_normalize(df_weather).apply(lambda x : x[0]['PCAT'])

In [5]:
t = t.T.reset_index().set_axis(['TCAT' for _ in range(t.shape[1])], axis = 0).T
t.columns = [t.columns, t.iloc[0]]
t = t[1:]

In [6]:
p = p.T.reset_index().set_axis(['PCAT' for _ in range(p.shape[1])], axis = 0).T
p.columns = [p.columns, p.iloc[0]]
p = p[1:]

In [7]:
t = t.merge(p, left_index=True, right_index=True)
a = (df_flight.fl_date.apply(lambda x : (x - datetime.timedelta(weeks = 1)).strftime('%Y%m%d')))
c = a[0:1].apply(lambda x : t.iloc[:, t.columns.get_level_values(1)==x])[0].merge(df_flight, left_index = True, right_on='origin_state')
c.columns = ['origin_TCAT', 'origin_PCAT'] + c.columns[2:].to_list()

/tmp/ipykernel_42551/355434365.py:3: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  c = a[0:1].apply(lambda x : t.iloc[:, t.columns.get_level_values(1)==x])[0].merge(df_flight, left_index = True, right_on='origin_state')


In [8]:
c = a[0:1].apply(lambda x : t.iloc[:, t.columns.get_level_values(1)==x])[0].merge(c, right_on='dest_state', left_index=True)
c.columns = ['dest_TCAT', 'dest_PCAT', 'origin_TCAT', 'origin_PCAT'] + c.columns[4:].to_list()
c

/tmp/ipykernel_42551/2419626230.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  c = a[0:1].apply(lambda x : t.iloc[:, t.columns.get_level_values(1)==x])[0].merge(c, right_on='dest_state', left_index=True)


,dest_TCAT,dest_PCAT,origin_TCAT,origin_PCAT,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,...,crs_dep_time_of_day,crs_arr_time_of_day,origin_state,dest_state,origin_region,dest_region,mean_mkt_carrier_delay,mean_op_carrier_delay,route,mean_route_delay
333,2,2,2,2,2018-10-19,AS,AS,AS,153,AS,...,late afternoon,late afternoon,AK,AK,West,West,0.711088,0.550692,OTZOME,-5.250000
388,2,2,2,2,2019-06-05,AS,AS,AS,50,AS,...,late afternoon,late afternoon,AK,AK,West,West,0.711088,0.550692,BRWANC,-7.473684
732,2,2,2,2,2018-09-28,AS,AS,AS,154,AS,...,late morning,late morning,AK,AK,West,West,0.711088,0.550692,OTZANC,4.266667
1123,2,2,2,2,2018-11-11,AS,AS,AS,70,AS,...,evening,evening,AK,AK,West,West,0.711088,0.550692,JNUSIT,-8.809524
1546,2,2,2,2,2018-09-14,AS,AS,AS,64,AS,...,midday,midday,AK,AK,West,West,0.711088,0.550692,JNUPSG,-0.888889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351147,2,2,0,2,2018-07-08,DL,DL,DL,1105,DL,...,midday,midday,UT,WY,West,West,2.359190,0.167413,SLCJAC,0.050847
353877,2,2,0,2,2019-06-22,DL,DL_CODESHARE,DL,4038,OO,...,late afternoon,late afternoon,UT,WY,West,West,2.359190,7.229355,SLCJAC,0.050847
358554,2,2,0,2,2019-08-18,DL,DL_CODESHARE,DL,4005,OO,...,late afternoon,late afternoon,UT,WY,West,West,2.359190,7.229355,SLCJAC,0.050847
360808,2,2,0,2,2019-08-13,DL,DL_CODESHARE,DL,4323,OO,...,midday,midday,UT,WY,West,West,2.359190,7.229355,SLCCOD,-9.157895


In [9]:
import pickle
flight_data_engineered6 = c[['dest_TCAT', 'dest_PCAT', 'origin_TCAT', 'origin_PCAT','fl_date','origin_state','dest_state']]
with open('../data/flight_train.pickle','wb') as weather_data_file:
     pickle.dump(flight_data_engineered6, weather_data_file)

In [10]:
flight_data_engineered6.shape

(352575, 7)

In [11]:
flight_data_engineered6

,dest_TCAT,dest_PCAT,origin_TCAT,origin_PCAT,fl_date,origin_state,dest_state
333,2,2,2,2,2018-10-19,AK,AK
388,2,2,2,2,2019-06-05,AK,AK
732,2,2,2,2,2018-09-28,AK,AK
1123,2,2,2,2,2018-11-11,AK,AK
1546,2,2,2,2,2018-09-14,AK,AK
...,...,...,...,...,...,...,...
351147,2,2,0,2,2018-07-08,UT,WY
353877,2,2,0,2,2019-06-22,UT,WY
358554,2,2,0,2,2019-08-18,UT,WY
360808,2,2,0,2,2019-08-13,UT,WY
